In [ ]:
# -*- coding: utf-8 -*-
"""
Project: Real Estate Handover Calculator (Expense List Edition)
Author: Gemini (AI Assistant)
Description: 
    - Header Update: "Paid To" renamed to "Tenant Paid To".
    - Detailed Expenses: Added specific rows for Cable/Internet/Deposits in the Buyer's Credit section.
    - Custom Inputs: Added 2 free-text rows for "Other Expenses".
    - Calculation Details: Retained the explicit "$Price x (Days/Denom)" text format.
    - Layout: Expanded the Summary section to accommodate the detailed expense list.
"""

import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side, Protection
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.worksheet.protection import SheetProtection
from openpyxl.utils import get_column_letter
import datetime


def generate_expense_list_excel():
    # ===================== 0. 初始化 =====================
    wb = Workbook()
    wb.remove(wb.active)
    ws = wb.create_sheet("交屋分算(費用明細)", 0)
    ws.sheet_view.showGridLines = False

    # ===================== 1. 美學設定 =====================
    # 標題區
    COLOR_HEADER_BG = "1E293B"    # Slate 800
    COLOR_HEADER_FONT = "FFFFFF"

    # 輸入區
    COLOR_INPUT_BG = "FFFFFF"

    # 賣方區 (Red)
    COLOR_SELLER_BG = "FFF1F2"
    COLOR_SELLER_HEADER = "E11D48"
    COLOR_SELLER_TXT = "881337"

    # 買方區 (Green)
    COLOR_BUYER_BG = "ECFDF5"
    COLOR_BUYER_HEADER = "059669"
    COLOR_BUYER_TXT = "064E3B"

    # 結算區
    COLOR_FINAL_BG = "FFEDD5"

    # 邊框
    COLOR_BORDER = "CBD5E1"
    COLOR_BORDER_THICK = "334155"

    font_title = Font(name="微軟正黑體", size=24, bold=True, color="1E293B")
    font_header_base = Font(name="微軟正黑體", size=10, bold=True, color="FFFFFF")

    font_normal = Font(name="微軟正黑體", size=10, color="334155")
    font_input = Font(name="微軟正黑體", size=11, color="000000")

    font_seller = Font(name="Arial", size=10, bold=True,
                       color=COLOR_SELLER_TXT)
    font_buyer = Font(name="Arial", size=10, bold=True, color=COLOR_BUYER_TXT)

    font_big_res = Font(name="Arial", size=20, bold=True, color="1E293B")
    font_hint = Font(name="微軟正黑體", size=9, color="64748B")

    border_std = Border(
        left=Side(style='thin', color=COLOR_BORDER),
        right=Side(style='thin', color=COLOR_BORDER),
        top=Side(style='thin', color=COLOR_BORDER),
        bottom=Side(style='thin', color=COLOR_BORDER)
    )

    border_thick_right = Border(
        left=Side(style='thin', color=COLOR_BORDER),
        right=Side(style='medium', color=COLOR_BORDER_THICK),
        top=Side(style='thin', color=COLOR_BORDER),
        bottom=Side(style='thin', color=COLOR_BORDER)
    )

    border_thick_bottom = Border(bottom=Side(style='medium', color="1E293B"))

    align_c = Alignment(horizontal="center", vertical="center", wrap_text=True)
    align_r = Alignment(horizontal="right", vertical="center")
    align_l = Alignment(horizontal="left", vertical="center", wrap_text=True)

    def set_cell(cell_ref, val, font=font_normal, fill=None, align=align_c, border=border_std, fmt=None):
        c = ws[cell_ref]
        c.value = val
        c.font = font
        c.alignment = align
        if fill:
            c.fill = PatternFill("solid", fgColor=fill)
        if border:
            c.border = border
        if fmt:
            c.number_format = fmt
        return c

    # ===================== 2. 欄位規劃 =====================
    # G欄名稱變更為「租客已繳至」
    cols = [
        ("A", 10), ("B", 10),
        ("C", 10), ("D", 10),
        ("E", 12), ("F", 12), ("G", 14),  # 稍微加寬
        ("H", 24), ("I", 12),
        ("J", 24), ("K", 12)
    ]
    for col_char, width in cols:
        ws.column_dimensions[col_char].width = width

    # ===================== 3. 儀表板頭部 =====================
    ws.merge_cells("A1:K1")
    set_cell("A1", "不動產交屋 · 租金分算結算表 (費用明細版)",
             font=font_title, align=align_l, border=None)

    for col in range(1, 12):
        ws.cell(2, col).border = border_thick_bottom

    # --- 關鍵參數 ---
    set_cell("A3", "交屋日期 (7碼)", font=font_header_base, fill=COLOR_HEADER_BG)
    set_cell("B3", 1140315, font=font_input, fill=COLOR_INPUT_BG, fmt="0")
    ws.merge_cells("C3:E3")
    verify_formula = f"""=IF(B3="","請輸入日期", "📅 " & TEXT(DATE(1911+LEFT(B3,3),MID(B3,4,2),RIGHT(B3,2)),"yyyy年mm月dd日"))"""
    set_cell("C3", verify_formula, font=font_hint, align=align_l, border=None)

    set_cell("A4", "交屋日算誰的", font=font_header_base, fill=COLOR_HEADER_BG)
    set_cell("B4", "歸買方", font=font_input, fill=COLOR_INPUT_BG)
    ws.merge_cells("C4:E4")
    set_cell("C4", '="⚖️ 權益切割：交屋日當天歸 " & RIGHT(B4,2)',
             font=font_hint, align=align_l, border=None)

    dv_owner = DataValidation(
        type="list", formula1='"歸買方,歸賣方"', allow_blank=False)
    ws.add_data_validation(dv_owner)
    dv_owner.add("B4:B4")

    set_cell("A5", "計算基準", font=font_header_base, fill=COLOR_HEADER_BG)
    set_cell("B5", "當月實際天數", font=font_normal, fill="F1F5F9")
    ws.merge_cells("C5:F5")
    days_in_month_formula = f"""=IF(B3="","", "📊 分母：依交屋當月總天數 (" & DAY(EOMONTH(DATE(1911+LEFT(B3,3),MID(B3,4,2),RIGHT(B3,2)),0)) & "天) 計算日租金")"""
    set_cell("C5", days_in_month_formula,
             font=font_hint, align=align_l, border=None)

    # ===================== 4. 核心數據表 =====================
    headers = [
        "房號/租客", "付款日\n(參考用)",
        "租期起", "租期訖",
        "月租金", "押金", "租客已繳至",  # 修改名稱
        "賣方補貼買方\n(計算詳情)", "補貼金額",
        "買方補貼賣方\n(計算詳情)", "補貼金額"
    ]

    for i, h in enumerate(headers):
        col = get_column_letter(i+1)
        bg_color = "64748B"
        txt_color = "FFFFFF"

        if "參考" in h:
            bg_color = "94A3B8"
        if "賣方" in h:
            bg_color = COLOR_SELLER_HEADER
        if "買方" in h:
            bg_color = COLOR_BUYER_HEADER

        # G欄右側加粗
        cell_border = border_std
        if col == "G":
            cell_border = border_thick_right

        set_cell(f"{col}7", h, font=font_header_base,
                 fill=bg_color, border=cell_border)

    start_row = 8
    end_row = 27

    # 付款日選單
    days_list = [str(i) for i in range(1, 32)] + ["月底"]
    formula_str = f'"{",".join(days_list)}"'
    dv_payment = DataValidation(
        type="list", formula1=formula_str, allow_blank=True)
    ws.add_data_validation(dv_payment)
    dv_payment.add(f"B{start_row}:B{end_row}")

    for r in range(start_row, end_row + 1):
        row_str = str(r)

        # 輸入區
        for c_char in ["A", "B", "C", "D", "E", "F"]:
            set_cell(f"{c_char}{row_str}", "", fill=COLOR_INPUT_BG)
        set_cell(f"G{row_str}", "", fill=COLOR_INPUT_BG,
                 border=border_thick_right)

        # 格式
        ws[f"B{row_str}"].number_format = "@"
        ws[f"C{row_str}"].number_format = "0"
        ws[f"D{row_str}"].number_format = "0"
        ws[f"E{row_str}"].number_format = "#,##0"
        ws[f"F{row_str}"].number_format = "#,##0"
        ws[f"G{row_str}"].number_format = "0"

        # 變數邏輯
        safe_b3 = f"IF($B$3=\"\", 1140101, $B$3)"
        d_handover = f"DATE(1911+LEFT({safe_b3},3),MID({safe_b3},4,2),RIGHT({safe_b3},2))"
        d_m_start = f"DATE(YEAR({d_handover}),MONTH({d_handover}),1)"
        d_m_end = f"EOMONTH({d_handover},0)"

        safe_g = f"IF(G{row_str}=\"\", {safe_b3}, G{row_str})"
        d_paid = f"DATE(1911+LEFT({safe_g},3),MID({safe_g},4,2),RIGHT({safe_g},2))"

        d_cut = f"IF($B$4=\"歸買方\", {d_handover}, {d_handover}+1)"
        denom_days = f"DAY({d_m_end})"

        # 權益計算
        entitled_days = f"MAX(0, {d_cut} - {d_m_start})"
        collected_days = f"MAX(0, MIN({d_paid}, {d_m_end}) - {d_m_start} + 1)"
        net_days = f"({collected_days} - {entitled_days})"

        is_valid = f'AND(A{row_str}<>"", E{row_str}<>"", G{row_str}<>"")'

        # --- 賣方補貼買方 ---
        s_date_start = d_cut
        s_date_end = f"MIN({d_paid}, {d_m_end})"
        s_desc_formula = f"""=IF({is_valid}, IF({net_days}>0, TEXT({s_date_start}, "mm/dd") & "~" & TEXT({s_date_end}, "mm/dd") & " (" & {net_days} & "天)" & CHAR(10) & "$" & TEXT(E{row_str}, "#,##0") & "×(" & {net_days} & "/" & {denom_days} & ")", ""), "")"""
        set_cell(f"H{row_str}", s_desc_formula,
                 fill=COLOR_SELLER_BG, font=font_seller)
        ws[f"H{row_str}"].alignment = Alignment(
            wrap_text=True, horizontal="center", vertical="center")

        s_amt_formula = f"""=IF({is_valid}, IF({net_days}>0, ROUND(E{row_str}/{denom_days}*{net_days}, 0), 0), "")"""
        set_cell(f"I{row_str}", s_amt_formula,
                 fill=COLOR_SELLER_BG, fmt="#,##0", font=font_seller)

        # --- 買方補貼賣方 ---
        b_date_start = f"MAX({d_m_start}, {d_paid}+1)"
        b_date_end = f"{d_cut}-1"
        b_desc_formula = f"""=IF({is_valid}, IF({net_days}<0, TEXT({b_date_start}, "mm/dd") & "~" & TEXT({b_date_end}, "mm/dd") & " (" & ABS({net_days}) & "天)" & CHAR(10) & "$" & TEXT(E{row_str}, "#,##0") & "×(" & ABS({net_days}) & "/" & {denom_days} & ")", ""), "")"""
        set_cell(f"J{row_str}", b_desc_formula,
                 fill=COLOR_BUYER_BG, font=font_buyer)
        ws[f"J{row_str}"].alignment = Alignment(
            wrap_text=True, horizontal="center", vertical="center")

        b_amt_formula = f"""=IF({is_valid}, IF({net_days}<0, ROUND(E{row_str}/{denom_days}*ABS({net_days}), 0), 0), "")"""
        set_cell(f"K{row_str}", b_amt_formula,
                 fill=COLOR_BUYER_BG, fmt="#,##0", font=font_buyer)

    # 範例
    set_cell("A8", "301 (範例)", fill=COLOR_INPUT_BG)
    set_cell("B8", "月底", fill=COLOR_INPUT_BG, align=align_c)
    set_cell("C8", 1130101, fill=COLOR_INPUT_BG, fmt="0")
    set_cell("D8", 1141231, fill=COLOR_INPUT_BG, fmt="0")
    set_cell("E8", 15000, fill=COLOR_INPUT_BG, fmt="#,##0")
    set_cell("F8", 30000, fill=COLOR_INPUT_BG, fmt="#,##0")
    set_cell("G8", 1140228, fill=COLOR_INPUT_BG,
             fmt="0", border=border_thick_right)

    # ===================== 5. 擴充結算區 (Expanded Summary) =====================
    sum_row = end_row + 2

    # ---------------- 左側：賣方應付 (Debits) ----------------
    ws.merge_cells(f"A{sum_row}:E{sum_row}")
    set_cell(f"A{sum_row}", "賣方應付項目 (Debits)", font=font_header_base,
             fill=COLOR_SELLER_HEADER, align=align_l)

    # 項目 1
    set_cell(f"A{sum_row+1}", "1. 押金移轉", align=align_l, border=None)
    set_cell(f"E{sum_row+1}", f"=SUM(F{start_row}:F{end_row})",
             fmt="#,##0", font=font_seller, align=align_r)

    # 項目 2
    set_cell(f"A{sum_row+2}", "2. 賣方補貼買方 (溢收租金)", align=align_l, border=None)
    set_cell(f"E{sum_row+2}", f"=SUM(I{start_row}:I{end_row})",
             fmt="#,##0", font=font_seller, align=align_r)

    # ---------------- 右側：買方應付 (Credits) - 新增費用清單 ----------------
    ws.merge_cells(f"G{sum_row}:K{sum_row}")
    set_cell(f"G{sum_row}", "買方應付項目 (Credits) - 含代墊費用",
             font=font_header_base, fill=COLOR_BUYER_HEADER, align=align_l)

    # 項目 1: 短收租金
    set_cell(f"G{sum_row+1}", "1. 買方補貼賣方 (短收租金)", align=align_l, border=None)
    set_cell(f"K{sum_row+1}", f"=SUM(K{start_row}:K{end_row})",
             fmt="#,##0", font=font_buyer, align=align_r)

    # 新增詳細費用清單 (2. ~ 8.)
    expenses_list = [
        "機上盒保證金",
        "有線電視費用",
        "網路費用",
        "監視器費用",
        "寬頻押金",
        "其他費用 (1)",  # 開放修改
        "其他費用 (2)"  # 開放修改
    ]

    start_exp_row = sum_row + 2

    for idx, item in enumerate(expenses_list):
        current_row = start_exp_row + idx
        # G欄顯示項目名稱 (如果是其他費用，設為可輸入)
        if "其他" in item:
            set_cell(f"G{current_row}", item, align=align_l,
                     border=None, fill=COLOR_INPUT_BG)  # 可輸入
        else:
            set_cell(f"G{current_row}", f"{idx+2}. {item}",
                     align=align_l, border=None)

        # K欄顯示金額 (可輸入)
        set_cell(f"K{current_row}", 0, fmt="#,##0",
                 font=font_buyer, fill=COLOR_INPUT_BG, align=align_r)

    # ---------------- 底部：Net Settlement ----------------
    final_row = start_exp_row + len(expenses_list) + 2
    ws.merge_cells(f"A{final_row}:D{final_row}")
    ws.merge_cells(f"E{final_row}:K{final_row}")

    # 總額計算：(押金+溢收) - (短收 + 所有雜項)
    # 雜項範圍: K{start_exp_row} 到 K{start_exp_row + len - 1}
    exp_range_sum = f"SUM(K{start_exp_row}:K{start_exp_row + len(expenses_list) - 1})"

    net_val = f"E{final_row}"
    ws[net_val] = f"=(E{sum_row+1}+E{sum_row+2}) - (K{sum_row+1} + {exp_range_sum})"

    ws[net_val].font = font_big_res
    ws[net_val].alignment = align_c
    ws[net_val].number_format = "#,##0"
    ws[net_val].fill = PatternFill("solid", fgColor=COLOR_FINAL_BG)

    text_formula = f"""=IF({net_val}>=0, "最終結算：賣方 應找補 買方", "最終結算：買方 應找補 賣方")"""
    set_cell(f"A{final_row}", text_formula, font=Font(
        size=14, bold=True, color="1E293B"), align=align_r, border=None)

    # ===================== 6. 保護 =====================
    ws.protection = SheetProtection(
        sheet=True, formatCells=False, formatColumns=False)

    unlock_ranges = [
        "B3:B4",
        f"A{start_row}:G{end_row}",
        f"K{start_exp_row}:K{start_exp_row + len(expenses_list) - 1}",  # 雜項金額
        f"G{start_exp_row + 5}:G{start_exp_row + 6}"  # 其他費用名稱
    ]

    for rng in unlock_ranges:
        for row in ws[rng]:
            for cell in row:
                cell.protection = Protection(locked=False)

    filename = f"交屋分算_費用明細版_{datetime.datetime.now().strftime('%H%M%S')}.xlsx"
    wb.save(filename)
    return filename


if __name__ == "__main__":
    print("生成費用明細版 Excel...")
    f = generate_expense_list_excel()
  print(f"完成！檔案：{f}")

SyntaxError: invalid non-printable character U+00A0 (590677811.py, line 23)